## 1. 写一个方法，实现从一个字串经过哪些操作后能得到另一个字串？

In [22]:
from functools import lru_cache
import copy

In [37]:
@lru_cache(maxsize=2**10) # 调用下面的函数时，若找到相同的输入，那么直接返回相同的输出结果
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2), []
    if len(string2) == 0: return len(string1), []
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    min_distance1, operations1 = edit_distance(string1[:-1], string2)
    min_distance1 += 1
    operations1 = copy.copy(operations1)
    operations1.append('DEL p{}_{}'.format(len(string1) - 1 , tail_s1),)  
    
    min_distance2, operations2 = edit_distance(string1, string2[:-1])
    operations2 = copy.copy(operations2)
    operations2.append('ADD p{}_{}'.format(len(string2) - 1 , tail_s2), )  
    min_distance2 += 1
    
    if tail_s1 == tail_s2:
        min_distance3, operations3 = edit_distance(string1[:-1], string2[:-1])
    else:
        min_distance3, operations3 = edit_distance(string1[:-1], string2[:-1])
        min_distance3 += 1
        operations3 = copy.copy(operations3)
        operations3.append('SUB p{}_{} => p{}_{}'.format(len(string1) - 1 ,tail_s1, len(string2) - 1, tail_s2))
    
    candidates = [
        (min_distance1, operations1),
        (min_distance2, operations2),
        (min_distance3, operations3),
    ] 
    
    min_distance, operations = min(candidates, key=lambda x: x[0])
    
    return min_distance, operations


In [38]:
edit_distance('ABCDECG','ABCCEF')

(3, ['SUB p3_D => p3_C', 'SUB p5_C => p5_F', 'DEL p6_G'])

In [39]:
edit_distance('特朗普称来自中国物资非常好','特朗普说中国物资都是垃圾')

(7,
 ['SUB p3_称 => p3_说',
  'DEL p4_来',
  'DEL p5_自',
  'SUB p10_非 => p8_都',
  'SUB p11_常 => p9_是',
  'SUB p12_好 => p10_垃',
  'ADD p11_圾'])

## 2. 如何让输入法输入字符串后在不带空格的时候完成修正？如何完成拼音的自动分割？

提示：使用第一节课提到的语言模型。

In [2]:
# import pinyin # 不知道为什么，安装后不能使用

先找个好用的拼音库：

In [ ]:
from xpinyin import Pinyin
pinyin = Pinyin()

In [11]:
pinyin.get_pinyin('我们是共产主义接班人')

# 默认以'-'为分割符
a = pinyin.get_pinyin(u"上海")
print(a)

# 默认以'-'分割,显示音调
b = pinyin.get_pinyin(u'上海', tone_marks='marks')
print(b)

# 默认以'-'分割,显示音调
b = pinyin.get_pinyin(u'上海', tone_marks='numbers')
print(b)

# 删除分割符
c = pinyin.get_pinyin(u'上海', '')
print(c)

# 设置空白格为分割符
d = pinyin.get_pinyin(u'上海', ' ')
print(d)

d1 = pinyin.get_initial(u'上')
print(d1)

d2 = pinyin.get_initials(u'上海')
print(d2)

d3 = pinyin.get_initials(u'上海', u'')
print(d3)

d4 = pinyin.get_initials(u'上海', u' ')
print(d4)

wordvalue = '上海'
s = pinyin.get_initials(wordvalue, u'').lower()
print(s)

# 多音字
print(pinyin.get_pinyin('重', tone_marks='numbers'))
print(pinyin.get_pinyin('和', tone_marks='numbers'))

shang-hai
shàng-hǎi
shang4-hai3
shanghai
shang hai
S
S-H
SH
S H
sh
zhong4
he2


In [62]:
from pypinyin import pinyin, lazy_pinyin, Style, load_phrases_dict, load_single_dict
from pypinyin.style import register

In [15]:
print(pinyin('你好'))  # [['nǐ'], ['hǎo']]
print(pinyin('中心中心', heteronym=True))  # 启用多音字模式  # [['zhōng', 'zhòng'], ['xīn']]
print(pinyin('中心中心', style=Style.FIRST_LETTER))  # 设置拼音风格，第一个字母 [['z'], ['x']]
print(pinyin('中心中心', style=Style.TONE2, heteronym=True))  # [['zho1ng', 'zho4ng'], ['xi1n']]
print(lazy_pinyin('中心'))  # 不考虑多音字的情况 # ['zhong', 'xin']

[['nǐ'], ['hǎo']]
[['zhōng', 'zhòng'], ['xīn'], ['zhōng', 'zhòng'], ['xīn']]
[['z'], ['x'], ['z'], ['x']]
[['zho1ng', 'zho4ng'], ['xi1n'], ['zho1ng', 'zho4ng'], ['xi1n']]
['zhong', 'xin']


In [18]:
##########处理不包含拼音的字符
# default (默认行为): 不做任何处理，原样返回:
print(lazy_pinyin('你好☆d☆你好'))  # ['ni', 'hao', '☆☆']
# ignore : 忽略该字符
print(lazy_pinyin('你好☆c☆你好', errors='ignore'))  # ['ni', 'hao']
# replace : 替换为去掉 \u 的 unicode 编码
print(lazy_pinyin('你好☆b☆你好', errors='replace'))  # ['ni', 'hao', '26062606']
# callable 对象 : 提供一个回调函数，接受无拼音字符(串)作为参数, 支持的返回值类型: unicode 或 list ([unicode, …]) 或 None 。
print(lazy_pinyin('你好☆a☆你好', errors=lambda x: 'star'))  # ['ni', 'hao', 'star']

########### 自定义拼音库
print(lazy_pinyin('还没', style=Style.TONE2))
load_phrases_dict({'桔子': [['jú'], ['zǐ']]})  # 增加 "桔子" 词组，可以自己定义
print(lazy_pinyin('桔子', style=Style.TONE2))

load_single_dict({ord('还'): 'hái,huán'})  # 调整 "还" 字的拼音顺序
print(lazy_pinyin('还没', style=Style.TONE2))

['ni', 'hao', '☆d☆', 'ni', 'hao']
['ni', 'hao', 'ni', 'hao']
['ni', 'hao', '2606622606', 'ni', 'hao']
['ni', 'hao', 'star', 'ni', 'hao']
['ha2i', 'me2i']
['ju2', 'zi3']
['ha2i', 'me2i']
['😘 me', '😘 me', '😘 dá']
['me', 'me', 'da']


In [ ]:
###########自定义拼音风格
@register('kiss')
def kiss(mypinyin, **kwargs):
    return '😘 {0}'.format(mypinyin)

print(lazy_pinyin('么么哒', style='kiss'))
print(lazy_pinyin('么么哒'))

还是pypinyin比较好，功能齐全！

接下来实现功能：

思路一： 训练时目标是构建一个字典，key是拼音，value是子字典（key是可能的词，value是 词库词频和用户输入次数等的数据对象），
训练时进行容错处理，应用时词选取根据词频和用户数据等信息计算后进行排序并显示给用户。

训练过程：

1. 用分词工具把语料切割成词，然后去重并统计每个词出现的次数。

2. 过滤掉非汉字字符。

3. 循环所有字和词的拼音以及容错性处理的拼音，建立这样一个字典。

4. 字典的持久化保存。

推理过程：

1. 如果用户输入的拼音能在字典中找到则直接计算出结果显示。

2. 如果用户输入的拼音太长则使用动态规划对拼音进行切割后执行1再拼接。

这一思路是本学生感觉最容易实现的一种方式。训练过程容易实现，容错容易处理，推理阶段计算实时高效，但内存可能消耗较大。

In [51]:
# 分词
import thulac
import pandas as pd
from collections import Counter
import numpy as np
import re

wordCuter = thulac.thulac(seg_only=True)

Model loaded succeed


In [20]:
def getCsvArticles(filename, contentColName, encoding='gb18030'):
    content = pd.read_csv(filename, encoding=encoding)
    articles = content[contentColName].tolist()
    return articles

content = pd.read_csv('../Lesson10/Data/新华社数据.csv', encoding='gb18030')
articles = content['content'].tolist()

In [37]:
# 分词过程
words = []
for i, article in enumerate(articles[:100]):
    if not isinstance(article, str):
        continue
    if i % 1000 == 999:
        print('{0}/{1}'.format(i + 1, len(articles)))
    _words = [ t1  for t1,t2 in wordCuter.cut(article, text=False)] 
    words += _words
    
wordsCounter = Counter(words)
print(wordsCounter)

Counter({'，': 3634, '的': 2471, '。': 1646, '\n': 976, '\r': 962, '、': 707, '在': 661, '了': 526, '是': 472, '不': 446, '和': 423, ' ': 381, '“': 369, '”': 369, '一': 333, '来': 298, '家': 239, '有': 218, ',': 208, '这': 207, '将': 204, '\u3000': 202, '：': 197, '也': 196, '为': 191, '上': 190, '中': 188, '.': 180, '大': 176, '人': 170, '对': 170, '个': 168, '等': 163, '到': 162, '市场': 155, '%': 153, '就': 147, '与': 144, '会': 136, '都': 136, '被': 129, '中国': 125, '从': 124, '多': 121, '并': 118, '我': 114, '还': 112, '后': 111, '股': 111, '两': 109, '下': 106, '我们': 103, '而': 100, '更': 97, '一个': 96, '年': 94, '新': 94, '要': 94, '6月': 92, '元': 92, '已经': 90, '说': 88, '能': 87, '他': 87, '以及': 87, '进行': 86, '次': 86, '（': 85, '）': 85, '记者': 85, '以': 84, '企业': 84, '但': 83, '未': 80, '银行': 80, '可以': 79, '内容': 79, '乐视': 78, '目前': 75, '时': 75, '手机': 74, '融资': 74, '仅': 72, '；': 72, '价': 71, '过': 71, '只': 70, '3': 70, '看': 70, '最': 69, '么': 69, '没有': 68, '让': 68, '国': 67, '或': 66, '4': 65, '可能': 65, '公司': 65, '其中': 64, '相关': 64, '前': 6

In [61]:
# 过滤掉非汉字字符

newWordsCounter = {}
for i, (word, wCount) in enumerate(wordsCounter.items()):
    # print(i, word, wCount)
    # re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+".encode('utf-8').decode('utf-8'), "".encode('utf-8').decode('utf-8'),line)
    # word = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+".encode('utf-8').decode('utf-8'), "", word)
    if i % 1000 == 999:
        print('{0}/{1}'.format(i + 1, len(wordsCounter)))

    _words = re.findall('[\u4e00-\u9fa5]+', word)
    if len(_words) == 0 : continue 
    for _word in _words:
        if _word != '':
            if _word in newWordsCounter:
                newWordsCounter[_word] += wCount
            else:
                newWordsCounter[_word] = wCount
            
print(newWordsCounter)

1000/8907
2000/8907
3000/8907
4000/8907
5000/8907
6000/8907
7000/8907
8000/8907
{'此外': 11, '自': 28, '本周': 2, '月': 183, '日': 163, '起': 47, '除': 16, '小米': 3, '手机': 74, '等': 163, '款': 24, '机型': 4, '外': 9, '其余': 3, '已': 49, '暂停': 4, '更新': 6, '发布': 26, '含': 4, '开发版': 1, '体验': 13, '版内测': 1, '稳定': 19, '版': 2, '暂': 3, '不': 446, '受': 18, '影响': 25, '以': 84, '确保': 6, '工程师': 5, '可以': 79, '集中': 7, '全部': 35, '精力': 4, '进行': 86, '系统': 23, '优化': 8, '工作': 54, '有人': 1, '猜测': 2, '这': 207, '也': 196, '是': 472, '将': 204, '主要': 34, '用': 38, '到': 162, '的': 2471, '研发': 12, '之中': 2, '去年': 20, '距': 1, '今': 8, '有': 218, '一': 333, '年': 217, '有余': 1, '时候': 28, '更新换代': 1, '了': 526, '当然': 9, '关于': 15, '确切': 1, '信息': 44, '我们': 103, '还是': 15, '等待': 5, '官方': 17, '消息': 35, '骁龙': 10, '作为': 31, '唯一': 5, '通过': 46, '桌面': 3, '平台': 48, '认证': 1, '处理器': 7, '高通': 4, '强调': 1, '会': 136, '因为': 29, '只': 70, '考虑': 12, '性能': 4, '而': 100, '去': 34, '屏蔽': 1, '掉': 6, '小': 44, '核心': 18, '相反': 3, '他们': 50, '正': 22, '联手': 5, '微软': 10, '找到': 9,

In [ ]:
# 循环所有字和词的拼音以及容错性处理的拼音，建立这样一个字典

class WordData:
    def __init__(self, word, baseCount):
        self.word = word
        self.baseCount = baseCount
        self.userUseCount = 0

model = {}
for i, (word, wCount) in enumerate(newWordsCounter.items()):
    # 词拼音
    wordPY = lazy_pinyin(word, errors='ignore')
    model[wordPY] = {word: WordData(word, wCount)}
    ...
    
    
    # 字拼音
    if len(word) == 0: continue
    for w in word:
        wpy = lazy_pinyin(w, errors='ignore')
        if wpy in model:
            model[wpy].baseCount += wCount
        else:
            model[wpy] = {w: WordData(w, wCount)}
        ...
    